In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/Machine Learning/Contest2019/AgeComparison')

FileNotFoundError: ignored

In [0]:
!pip install keras_vggface
import numpy as np
import tensorflow as tf
import random
from keras.models import Model
from keras import backend as K
from keras_vggface import VGGFace
from keras.optimizers import Adam
from keras.layers import Concatenate, Dropout,GlobalAveragePooling2D, Input,BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Input, Lambda
from keras.models import Model, Sequential
from keras.regularizers import l2
import keras
from keras_vggface import VGGFace

In [0]:
def get_model(input_shape=(192, 192, 3)):
    #pre_trained=InceptionV3(weights="imagenet")
    resize = keras.layers.Lambda( 
    lambda image: tf.image.resize_images( 
            image, 
            (224, 224), 
            method = tf.image.ResizeMethod.BICUBIC,
            align_corners = True, # possibly important
            preserve_aspect_ratio = True
        )
    )

    inp = Input(shape=(192, 192, 3))
    res = resize(inp)
    # add a global spatial average pooling layer
    base_model = VGGFace(input_tensor=res, pooling='avg')
    x = base_model.layers[-7].output    
    normalize = Lambda(lambda x: K.l2_normalize(x, axis=-1), name='normalize')
    x=normalize(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    model = Model(inputs=inp, outputs=x)

    return model

base_model = get_model()

xa_inp = Input(shape=(192, 192, 3))
xb_inp = Input(shape=(192, 192, 3))
x1=base_model(xa_inp)
x2=base_model(xb_inp)
out = Concatenate()([x1, x2])
out = Dense(3, activation="softmax")(out)
model = Model(inputs=[xa_inp,xb_inp], outputs=out)


opt = Adam(lr=1e-5)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [0]:
import h5py

f=h5py.File('train_set_shuffled.h5', 'r')
# NOTE! We don't make a in-memory copy of the dataset... So we cannot close the file!

X1=f['X1']   # Tensor with first images in the pair
X2=f['X2']   # Tensor with second images in the pair
y=f['y']     # Tensor with desired output


print(X1.shape, X2.shape, y.shape)

import numpy as np

train = 0.80 
test = 0.10
val = 0.10

#Splitting 
x1_val = X1[:int(X1.shape[0]*val)]
x2_val = X2[:int(X1.shape[0]*val)]
y_val = y[:int(X1.shape[0]*val)]
print("Done val")
x1_train = X1[int(X1.shape[0]*val):]
x2_train = X2[int(X1.shape[0]*val):]
y_train = y[int(X1.shape[0]*val):] 
print("Done train")


In [0]:
#Generators
import numpy as np
from PIL import Image, ImageEnhance
from keras.preprocessing.image import img_to_array, array_to_img
import random

def random_brightness(x, interval=[0.5, 1.5]):        
    apply = np.random.randint(0,2)
    if apply:
        value = random.uniform(interval[0],interval[1])
        img = array_to_img(x)
        enhancer = ImageEnhance.Brightness(img)
        enhancer.enhance(value)
        x = img_to_array(img)    
    return x
    

def generate_val(batch_size):
    x1_batch, x2_batch, y_batch = [],[],[]
    
    while len(x1_batch)<batch_size:
        index = np.random.randint(0,x1_val.shape[0])
        x1, x2, y = x1_val[index], x2_val[index], y_val[index]
        x1_batch.append(x1)
        x2_batch.append(x2)
        y_batch.append(y)
        
    return [np.array(x1_batch)/255.0, np.array(x2_batch)/255.0], np.array(y_batch)
    

def generate_train(batch_size):
    x1_batch, x2_batch, y_batch = [],[],[]
    left, center, right = 0, 0, 0
    
    while len(x1_batch)<batch_size:
        index = np.random.randint(0,x1_train.shape[0])
        
        #Prendo, nel batch, un numero uguale di classi
        if y_train[index][0] and left >= batch_size//3:
            continue
        if y_train[index][1] and center >= batch_size//3:
            continue
        if y_train[index][2] and right >= batch_size//3:
            continue
        
        x1, x2, y = x1_train[index], x2_train[index], y_train[index]
        x1=random_brightness(x1)
        x2=random_brightness(x2)
        
        x1_batch.append(x1)
        x2_batch.append(x2)
        y_batch.append(y)
        
        x1, x2 = np.flip(x1, axis=1), np.flip(x2,axis=1)
        x1=random_brightness(x1)
        x2=random_brightness(x2)
        
        x1_batch.append(x1)
        x2_batch.append(x2)
        y_batch.append(y)
        
        if y[0]:
            left+=2
        elif y[1]:
            center+=2
        else:
            right+=2
    return [np.array(x1_batch)/255.0, np.array(x2_batch)/255.0], np.array(y_batch)



def generate_train_batch(batch_size):
    while True:
        yield generate_train(batch_size)

def generate_val_batch(batch_size):
    while True:
        yield generate_val(batch_size)
    
for (x1,x2),y in generate_val_batch(17):
    print(x1.shape)
    print(np.sum(y, axis=0))
    break

In [0]:
import keras

filepath = "siamese.{epoch:02d}-{val_loss:.2f}.hdf5"

callbacks= [ keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=10)]

In [0]:
batch_size=33

nb_train = y_train.shape[0]/batch_size
nb_validation = y_val.shape[0]/batch_size
history = model.fit_generator(
                    generate_train_batch(batch_size),
                    steps_per_epoch=nb_train,
                    epochs=100,
                    validation_data=generate_val_batch(batch_size),
                    validation_steps=nb_validation,
                    verbose=1, callbacks=callbacks)